## 과제 1
ReLu activation function과 derivative function을 구현해보세요
- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [6]:
import numpy as np

In [7]:
def relu(x):
  return np.maximum(0,x)

In [8]:
def d_relu(x):
  d_relu = lambda x: 0 if x<0 else 1
  return np.array(list(map(d_relu, x)))

In [9]:
def d_relu(x):
  x[x>=0] = 1
  x[x<0] = 0
  return x

## 과제 2
Deep Learning Basic 코드 파일의 MLP implementation with Numpy library using MNIST dataset 코드 참고해서
Three layer MLP 일 때의 backward_pass 함수를 완성해주세요.   
- Hint : 코드 파일의 예시는 Two layer MLP


In [ ]:
def backward_pass(x, y_true, params):

  dS3 = params["A3"] - y_true
 
  grads = {}

  grads["dW3"] =  np.dot(dS3, params["A2"].T)/x.shape[1]
  grads["db3"] =  (1/x.shape[1])*np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

  dA2 = np.dot(params["W3"].T, dS3)
  dS2 = dA2 * d_sigmoid(params["S2"])

  grads["dW2"] =  np.dot(dS2, params["A1"].T)/x.shape[1]
  grads["db2"] =  (1/x.shape[1])*np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1 = np.dot(params["W2"].T, dS2)
  dS1 = dA1 * d_sigmoid(params["S1"])

  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]

  return grads

## 과제 3
Deep Learning Basic 코드 파일의 MLP implementation with Pytorch library using MNIST dataset 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [10]:
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 이미지를 텐서로 변경
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

BATCH_SIZE = 512
# train set과 test set 각각에 대하여 DataLoader를 생성합니다.
# shuffle=True 매개변수를 넣어 데이터를 섞어주세요.
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw



In [11]:
class Net(nn.Module): # nn.Module <- torch.nn에 있는 클래스인 것 같다. 우리는 이 클래스를 상속받는 Net 클래스를 따로 만든다.
    def __init__(self):
        super(Net, self).__init__() # nn.Module의 __init__를 호출하는 함수
        self.layer1 = nn.Linear(784,128) # 28x28x1(흑백)=784, 128(=노드갯수)
        self.layer2 = nn.Linear(128,64) # 128 -> 64
        self.layer3 = nn.Linear(64,10) # 0~9 = 10개 <- 결과값

        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x) # layer1
        out = self.relu(out) # 활성화함수
        out = self.layer2(out) # layer2
        out = self.relu(out) # 활성화함수
        out = self.layer3(out) # layer3


        return out # out = w3 * max(0, w2 * max(0, w1 * x))

model = Net()
model

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

def train(model, train_loader, optimizer):
    # 모델을 훈련 모드로 전환
    model.train()
    # 배치 당 loss 값을 담을 리스트 생성
    batch_losses = []

    for data, target in train_loader:
        # 옵티마이저의 기울기 초기화 <- train에서만 수행(1)
        optimizer.zero_grad()

        # y pred 값 산출
        output = model(data)
        # loss 계산
        # 정답 데이터와의 cross entropy loss 계산
        # 이 loss를 배치 당 loss로 보관
        loss = criterion(output, target) # output(y-pred; 예측)과 target(정답)을 비교
        batch_losses.append(loss)

        # 기울기 계산
        loss.backward() # <- train에서만 수행(2) 아마도 backward함수를 직접 구현하는 것이 아니라, 이미 있는 것을 사용하는 것 같다.

        # 가중치 업데이트!
        optimizer.step()
        
    # 배치당 평균 loss 계산
    avg_loss = sum(batch_losses) / len(batch_losses)
    
    return avg_loss
  
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    batch_losses = []
    correct = 0 

    # 평가 과정에서는 기울기의 계산이 없다.
    with torch.no_grad(): 
        for data, target in test_loader:
            # 예측값 생성
            output = model(data)

            # loss 계산 (이전과 동일)
            loss = criterion(output, target)
            batch_losses.append(loss)

           # Accuracy 계산
           # y pred와 y가 일치하면 correct에 1을 더해주기
            pred = output.max(1, keepdim=True)[1] # 가장 큰 확률을 가진 것이 정답으로 설정

            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # 배치 당 평균 loss 계산 
    avg_loss =  sum(batch_losses) / len(batch_losses)

    #정확도 계산
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

EPOCHS = 100 # 100번

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.8450	Test Loss: 0.3295	Accuracy: 89.94%
[2] Train Loss: 0.2363	Test Loss: 0.3546	Accuracy: 88.54%
[3] Train Loss: 0.1745	Test Loss: 0.1946	Accuracy: 93.92%


KeyboardInterrupt: ignored

## 과제 4
과제 3 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting


1. hyperparameter는 다음과 같이 설정

- epochs : 10,50,100,500
- hiddensize<br>
[2-layer : (784,64,10),(784,512,10)]<br>
[3-layer : (784,512,128,10),(784,256,64,10)]<br>
[10-layer : (784,628,512,410,320,256,128,64,32,16,10),(784,700,690,680,670,660,650,640,630,620,10)]<br>
- learning_rate : 0.01,0.5,0.9

2. Layer 수 : 2,3,10
3. Activation Function : Relu, Sigmoid, Step, Leakly-Relu

In [12]:
epochs = [10,50,100,500]
hiddensize = {2:[(784,64,10),(784,512,10)],
              3:[(784,512,128,10),(784,256,64,10)],
             10:[(784,628,512,410,320,256,128,64,32,16,10),(784,700,690,680,670,660,650,640,630,620,10)]}
learning_rate = [0.01,0.5,0.9]
# act_fun = [relu, leakltyRelu, sigmoid, step]

class Net(nn.Module): # nn.Module <- torch.nn에 있는 클래스인 것 같다. 우리는 이 클래스를 상속받는 Net 클래스를 따로 만든다.
    def __init__(self, layer_size): # layer_size <- ex.(784,128,64,10) = 3-layers
        super(Net, self).__init__() # nn.Module의 __init__를 호출하는 함수

        self.layer_size = layer_size
        self.layers = []
        for i in range(len(layer_size)-1):
          self.layers.append(nn.Linear(layer_size[i],layer_size[i+1]))
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        for i in range(len(self.layer_size)-1):
          if i == 0:
            out = self.layers[i](x)
          else:
            out = self.relu(out)
            out = self.layers[i](out)
        return out


for idx1 in range(len(epochs)):
  for idx2 in range(len(hiddensize.keys())):
    for idx3 in range(len(hiddensize[list(hiddensize.keys())[idx2]])):
      for idx4 in range(len(learning_rate)):

        model = Net(hiddensize[list(hiddensize.keys())[idx2]][idx3]) # layer_size <- ex.(784,128,64,10)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate[idx4])

        EPOCHS = epochs[idx1]
        
        for epoch in range(1, EPOCHS + 1):
          train_loss = train(model, train_loader, optimizer)
          test_loss, test_accuracy = evaluate(model, test_loader)
          if epochs == EPOCHS:
            print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
              epoch, train_loss, test_loss, test_accuracy))

ValueError: ignored

layers를 바꾸는 것에 한계가 있어서, layers수를 3개로 고정시키고 hidden layer의 node수를 바꿔보자.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

epochs = [10,50] # 100,500도 하려했으나.. 시간이 너무 걸림으로 취소..
node_size_interval = [1,2,5,10,100,200,300] # 레이어간 노드 갯수 차이 ex. 300 = 784,484,184,10
learning_rate = [0.01,0.03,0.1,0.5,0.9]

result = pd.DataFrame() # 결과를 저장할 장소

class Net(nn.Module):
    def __init__(self, node_step):
        super(Net, self).__init__()
        self.node_step = node_step
        self.layer1 = nn.Linear(784,784-self.node_step)
        self.layer2 = nn.Linear(784-self.node_step,784-self.node_step*2)
        self.layer3 = nn.Linear(784-self.node_step*2,10)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x) # layer1
        out = self.relu(out) # 활성화함수
        out = self.layer2(out) # layer2
        out = self.relu(out) # 활성화함수
        out = self.layer3(out) # layer3

        return out

for i in epochs:
  for j in node_size_interval:
    for k in learning_rate:

      model = Net(j)

      starting_time = time.time()
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.SGD(model.parameters(), lr=k)
      EPOCHS = i

      for epoch in range(1, EPOCHS + 1):
          train_loss = train(model, train_loader, optimizer)
          test_loss, test_accuracy = evaluate(model, test_loader)
          if epoch == EPOCHS:
            result = pd.concat([result,pd.DataFrame(np.array([i,j,k,time.time()-starting_time,test_accuracy]).reshape(1,-1))])
            print('epochs: {}, node_size_interval: {}, learning_rate: {}, time: {:.2f}, accuracy: {:.2f}%'.format(
                i,j,k,time.time()-starting_time,test_accuracy))

result.columns=['epochs','node_size_interval','learning_rate','time','accuracy']
result.to_csv("/content/drive/MyDrive/result.csv",sep=',',index=False, header=False)

epochs: 10, node_size_interval: 1, learning_rate: 0.01, time: 117.36, accuracy: 87.32%
epochs: 10, node_size_interval: 1, learning_rate: 0.03, time: 117.45, accuracy: 91.39%
epochs: 10, node_size_interval: 1, learning_rate: 0.1, time: 116.09, accuracy: 94.41%
epochs: 10, node_size_interval: 1, learning_rate: 0.5, time: 114.08, accuracy: 97.21%
epochs: 10, node_size_interval: 1, learning_rate: 0.9, time: 112.21, accuracy: 97.51%
epochs: 10, node_size_interval: 2, learning_rate: 0.01, time: 111.79, accuracy: 86.93%
epochs: 10, node_size_interval: 2, learning_rate: 0.03, time: 111.40, accuracy: 91.19%
epochs: 10, node_size_interval: 2, learning_rate: 0.1, time: 111.56, accuracy: 94.74%
epochs: 10, node_size_interval: 2, learning_rate: 0.5, time: 111.08, accuracy: 97.86%
epochs: 10, node_size_interval: 2, learning_rate: 0.9, time: 111.12, accuracy: 98.08%
epochs: 10, node_size_interval: 5, learning_rate: 0.01, time: 110.05, accuracy: 87.47%
epochs: 10, node_size_interval: 5, learning_rate:

KeyboardInterrupt: ignored

**무엇을 보완하였고, 왜 보완되었는지에 대한 자유 서술 (아래에)**

---
(1) layer의 노드 수 (2) epochs (3) learning_rate를 변경해보았다.

1. learning_rate가 높아질수록 걸리는 시간은 짧아지고 accuracy는 높아진다.
2. 노드수가 줄어들수록 걸리는 시간은 상대적으로 많이 짧아지나 accuracy는 상대적으로 조금 감소한다.
3. epochs가 증가할수록 time은 상대적으로 매우 많이 증가하나 accuracy는 상대적으로 매우 조금 증가한다.

잘 보면 시간은 크게 영향을 받는것에 비해 accuracy는 조금 변화가 이뤄지는 것 같다.